In [1]:
import pandas as pd
import numpy as np

In [2]:
data_preprocessed=pd.read_csv("df_preprocessed.csv")

In [3]:
data_preprocessed.head()


,Reason_0,Reaseon_1,Reason_2,Reason_3,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


### Forming the target

In [4]:
target=np.where(data_preprocessed['Absenteeism Time in Hours']>data_preprocessed['Absenteeism Time in Hours'].median(),1,0)

In [5]:
target

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,

In [6]:
data_preprocessed["Exessive Absenteeism"]=target

In [7]:
data_with_target=data_preprocessed.drop(['Absenteeism Time in Hours','Day of the Week',
                                         'Distance to Work', 'Daily Work Load Average'] ,axis=1)

In [8]:
data_with_target is data_preprocessed

False

In [9]:
target.sum()/target.shape[0]

0.45571428571428574

## Selecting the input 

In [10]:
unscaled_target=data_with_target.iloc[:,:-1]

## Standardizing the input

In [11]:
#from sklearn.preprocessing import StandardScaler
#absenteeism_scaler=StandardScaler()

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler


In [13]:
class CustomScaler(BaseEstimator,TransformerMixin):
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler=StandardScaler(copy,with_mean,with_std)
        self.columns=columns
        self.mean=None
        self.var=None
    def fit(self,X,y=None):
        self.scaler.fit(X[self.columns],y)
        self.mean_=np.mean(X[self.columns])
        self.var_=np.var(X[self.columns])
        return self
    def transform(self,X,y=None,copy=None):
        init_col_order=X.columns
        X_scaled=pd.DataFrame(self.scaler.transform(X[self.columns]),columns=self.columns)
        X_not_scaled=X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled,X_scaled],axis=1)[init_col_order]

In [14]:
unscaled_target.columns.values

array(['Reason_0', 'Reaseon_1', 'Reason_2', 'Reason_3', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [15]:
columns_to_scale=['Month Value', 'Transportation Expense',
       'Age', 'Body Mass Index',
       'Children', 'Pets']

In [16]:
absenteeism_scaler=CustomScaler(columns_to_scale)

In [17]:
absenteeism_scaler.fit(unscaled_target)

C:\Users\Jubair\Anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'],
             copy=None, with_mean=None, with_std=None)

In [18]:
scaled_input=absenteeism_scaler.transform(unscaled_target)

In [19]:
scaled_input

,Reason_0,Reaseon_1,Reason_2,Reason_3,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [20]:
scaled_input.shape

(700, 11)

## Spliting the data

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x_train, x_test, y_train, y_test=train_test_split(scaled_input,target,train_size=.8,random_state=10)

In [23]:
x_train.shape

(560, 11)

In [24]:
y_train.shape

(560,)

## LogisticRegression Model

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [26]:
reg_model=LogisticRegression()

In [27]:
reg_model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
reg_model.score(x_train,y_train)

0.7785714285714286

## Manually testing

In [29]:
output=reg_model.predict(x_train)

In [30]:
output

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,

In [31]:
output==y_train

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False,  True, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,

In [32]:
np.sum(output==y_train)/y_train.shape

array([0.77857143])

## Finding the cofficient and the intercept

In [33]:
reg_model.intercept_

array([-1.82263588])

In [34]:
reg_model.coef_


array([[ 2.93520919,  0.84178219,  3.27310512,  1.07049903,  0.05411148,
         0.62119897, -0.27969801,  0.21371833,  0.17195311,  0.4488227 ,
        -0.32672004]])

In [35]:
summary_table=pd.DataFrame(columns=['Feature Name'],data=unscaled_target.columns.values)

summary_table['Coefficient']=np.transpose(reg_model.coef_)

In [36]:
summary_table

,Feature Name,Coefficient
0,Reason_0,2.935209
1,Reaseon_1,0.841782
2,Reason_2,3.273105
3,Reason_3,1.070499
4,Month Value,0.054111
5,Transportation Expense,0.621199
6,Age,-0.279698
7,Body Mass Index,0.213718
8,Education,0.171953
9,Children,0.448823


In [37]:
summary_table.index = summary_table.index + 1
summary_table.loc[0]=['Intercept',reg_model.intercept_[0]]
summary_table=summary_table.sort_index()
summary_table

,Feature Name,Coefficient
0,Intercept,-1.822636
1,Reason_0,2.935209
2,Reaseon_1,0.841782
3,Reason_2,3.273105
4,Reason_3,1.070499
5,Month Value,0.054111
6,Transportation Expense,0.621199
7,Age,-0.279698
8,Body Mass Index,0.213718
9,Education,0.171953


## Interpriting the coefficient  

In [38]:
summary_table['Odds_ratio']=np.exp(summary_table.Coefficient)

In [39]:
#log(odds) is used to interpret the significance of the coefficient. The value of log(odds) of the coefficient close to 1 
# are less significent where as others are most significant.

In [40]:
summary_table.sort_values('Odds_ratio',ascending=False)

,Feature Name,Coefficient,Odds_ratio
3,Reason_2,3.273105,26.393166
1,Reason_0,2.935209,18.825441
4,Reason_3,1.070499,2.916835
2,Reaseon_1,0.841782,2.320499
6,Transportation Expense,0.621199,1.861158
10,Children,0.448823,1.566467
8,Body Mass Index,0.213718,1.238274
9,Education,0.171953,1.187622
5,Month Value,0.054111,1.055602
7,Age,-0.279698,0.756012


## Testing the model with test data

In [41]:
reg_model.score(x_test,y_test)

0.7214285714285714

In [42]:
reg_model.predict(x_test)

array([0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0])

In [43]:
predicted_probability=reg_model.predict_proba(x_test)

In [44]:
predicted_probability

array([[0.81334354, 0.18665646],
       [0.78336556, 0.21663444],
       [0.39560298, 0.60439702],
       [0.2925537 , 0.7074463 ],
       [0.59333627, 0.40666373],
       [0.69339811, 0.30660189],
       [0.75884248, 0.24115752],
       [0.35554083, 0.64445917],
       [0.71589706, 0.28410294],
       [0.35908862, 0.64091138],
       [0.80163081, 0.19836919],
       [0.68679017, 0.31320983],
       [0.25948327, 0.74051673],
       [0.25948327, 0.74051673],
       [0.44066666, 0.55933334],
       [0.94989738, 0.05010262],
       [0.79162044, 0.20837956],
       [0.37728996, 0.62271004],
       [0.3949056 , 0.6050944 ],
       [0.69667275, 0.30332725],
       [0.32503624, 0.67496376],
       [0.86305309, 0.13694691],
       [0.25928813, 0.74071187],
       [0.19628316, 0.80371684],
       [0.06168528, 0.93831472],
       [0.13748188, 0.86251812],
       [0.06289224, 0.93710776],
       [0.49782338, 0.50217662],
       [0.87189303, 0.12810697],
       [0.34849298, 0.65150702],
       [0.

In [45]:
predicted_probability[:,0]

array([0.81334354, 0.78336556, 0.39560298, 0.2925537 , 0.59333627,
       0.69339811, 0.75884248, 0.35554083, 0.71589706, 0.35908862,
       0.80163081, 0.68679017, 0.25948327, 0.25948327, 0.44066666,
       0.94989738, 0.79162044, 0.37728996, 0.3949056 , 0.69667275,
       0.32503624, 0.86305309, 0.25928813, 0.19628316, 0.06168528,
       0.13748188, 0.06289224, 0.49782338, 0.87189303, 0.34849298,
       0.78336556, 0.78336556, 0.76445165, 0.87701674, 0.61914487,
       0.51721043, 0.17991219, 0.11192875, 0.80163081, 0.76722218,
       0.71589706, 0.32775284, 0.17070665, 0.31727281, 0.78856328,
       0.71902877, 0.12689788, 0.49550073, 0.86666429, 0.76886539,
       0.7616584 , 0.33804508, 0.48777836, 0.78336556, 0.86662631,
       0.33423631, 0.39121995, 0.87364504, 0.60076974, 0.76722218,
       0.3949056 , 0.51721043, 0.60446937, 0.25771457, 0.86843949,
       0.04845396, 0.26526646, 0.0477466 , 0.69282507, 0.70616609,
       0.68679017, 0.70616609, 0.38139267, 0.60836954, 0.30223

In [46]:
predicted_probability[:,1]

array([0.18665646, 0.21663444, 0.60439702, 0.7074463 , 0.40666373,
       0.30660189, 0.24115752, 0.64445917, 0.28410294, 0.64091138,
       0.19836919, 0.31320983, 0.74051673, 0.74051673, 0.55933334,
       0.05010262, 0.20837956, 0.62271004, 0.6050944 , 0.30332725,
       0.67496376, 0.13694691, 0.74071187, 0.80371684, 0.93831472,
       0.86251812, 0.93710776, 0.50217662, 0.12810697, 0.65150702,
       0.21663444, 0.21663444, 0.23554835, 0.12298326, 0.38085513,
       0.48278957, 0.82008781, 0.88807125, 0.19836919, 0.23277782,
       0.28410294, 0.67224716, 0.82929335, 0.68272719, 0.21143672,
       0.28097123, 0.87310212, 0.50449927, 0.13333571, 0.23113461,
       0.2383416 , 0.66195492, 0.51222164, 0.21663444, 0.13337369,
       0.66576369, 0.60878005, 0.12635496, 0.39923026, 0.23277782,
       0.6050944 , 0.48278957, 0.39553063, 0.74228543, 0.13156051,
       0.95154604, 0.73473354, 0.9522534 , 0.30717493, 0.29383391,
       0.31320983, 0.29383391, 0.61860733, 0.39163046, 0.69776

## Pickling Data

In [47]:
import pickle

In [48]:
with open('model','wb') as file:
    pickle.dump(reg_model,file)

In [49]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)